## Final training and testing using the full MNIST dataset

In [1]:
from sklearn.datasets import fetch_openml, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from copy import deepcopy
import seaborn as sns
import itertools
import copy
import autograd.numpy as np
import pandas as pd
from tqdm import tqdm

from activation_functions import *
from cost_functions import *
from FFNN import *

# Setting the random seed
np.random.seed(42)

In [ ]:
# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
# Extract data (features) and target (labels)
X = mnist.data
y = mnist.target

# Scaling the mnist pixel values from 0-255 to 0-1
X = X / 255.0

(70000, 784)
(70000,)


In [3]:
# Splitting the data into testing, training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# One-hot encoding
y_train = to_one_hot(y_train)
y_test = to_one_hot(y_test)


In [ ]:
# Define parameters
layer_output_size = [1000, 10]
num_hidden_layers = 1
eta = 0.01
activation_funcs = [sigmoid, softmax]
epochs = 100
optimizers = [train_network_SRMSprop]

network_input_size = X_train.shape[1]

In [6]:
# Create network
NN = NeuralNetwork(network_input_size, layer_output_size, activation_funcs, cross_entropy)


In [ ]:
# Training network
train_network_SRMSprop(NN, X_train, y_train, eta=eta, epochs=epochs)